In [ ]:
from keras.datasets import mnist
import numpy as np
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
import math
import matplotlib.pyplot as plt

Load Data, Generate noisy images and normalize

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

#make noisy images by flipping 0.03*no. of pixels per image
noise_factor = 0.03
total=28*28
#generate floor(0.03*28*28) pixel indices to flip
temp_noisy1=np.copy(x_train)
temp_noisy2=np.copy(x_train)
x_test_noisy=np.copy(x_test)
for image in range(0,len(temp_noisy1)):
    indices1=np.random.random_integers(0,total-1,size=int(total*noise_factor))
    indices2 = np.random.random_integers(0, total - 1, size=int(total * noise_factor))
    for i in indices1:
        temp_noisy1[image,i/28,i%28]=255-temp_noisy1[image,i/28,i%28]
    for i in indices2:
        temp_noisy2[image, i / 28, i % 28] = 255 - temp_noisy2[image, i / 28, i % 28]
x_train_noisy = np.concatenate((temp_noisy1,temp_noisy2),axis=0)

for image in range(0, len(x_test_noisy)):
    indices3 = np.random.random_integers(0, total - 1, size=int(total * noise_factor))
    for i in indices3:
        x_test_noisy[image, i / 28, i % 28] = 255 - x_test_noisy[image, i / 28, i % 28]

#normalize the pixel values
x_train_noisy = x_train_noisy.astype('float32') / 255.
x_test_noisy = x_test_noisy.astype('float32') / 255.
x_train= x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

Method to plot best and worst 10 digits based on RMSE

In [ ]:
def plot_digits(x_test,decoded_imgs):
    total=28*28
    #to calculate RMSE of each image for the final predicted op
    RMSE=np.zeros(len(x_test))
    temp=np.square(x_test-decoded_imgs)
    for i in range(0,len(x_test)):
        RMSE[i]=math.sqrt(np.sum(temp[i])/total)
    sorted_RMSE_indices=RMSE.argsort()
    best_10=sorted_RMSE_indices[:10]
    worst_10=sorted_RMSE_indices[-10:][::-1]

    print ("BEST 10 are indices are: ",best_10)
    print ("WORST 10 are indices are: ",worst_10)

    #plot best and worst 10 predictions
    #plot best and worst 10 predictions
    n = 10  # how many digits we will display
    plt.figure()
    for i in range(n):
        # display best original
        ax = plt.subplot(4, n, i + 1)
        plt.imshow(x_test_noisy[best_10[i]].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        #display best reconstructed
        ax = plt.subplot(4, n, i + 1+n)
        plt.imshow(decoded_imgs[best_10[i]].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display worst original
        ax = plt.subplot(4, n, i + 1+2*n)
        plt.imshow(x_test_noisy[worst_10[i]].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        #display worst reconstructed
        ax = plt.subplot(4, n, i + 1 + 3*n)
        plt.imshow(decoded_imgs[worst_10[i]].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.suptitle("top rows = 10 best digits (original,reconstructed) \n bottom rows = 10 worst digits (original,reconstructed)")
    plt.show()

Autoencoder- Architecture 1

In [ ]:
#ARCHITECTURE 1 - As in blog

input_img = Input(shape=(28, 28, 1))  

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (7, 7, 32)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy',metrics=['mse'])

autoencoder.fit(x_train_noisy, np.concatenate((x_train,x_train),axis=0),
                epochs=12, 
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/tb', histogram_freq=0, write_graph=False)])
decoded_imgs = autoencoder.predict(x_test_noisy)

plot_digits(x_test,decoded_imgs)


Plot- Autoencoder 1
<img src="Arch1.png">

Autoencoder- Architecture 2: Changing number of filters from 32 to 16 in each layer

In [ ]:
#ARCHITECTURE 2:Changing number of filters from 32 to 16 in each layer

input_img = Input(shape=(28, 28, 1))  

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy',metrics=['mse'])

autoencoder.fit(x_train_noisy, np.concatenate((x_train,x_train),axis=0), 
                epochs=12, 
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/tb', histogram_freq=0, write_graph=False)])
decoded_imgs = autoencoder.predict(x_test_noisy)

plot_digits(x_test,decoded_imgs)

Plot- Autoencoder 2
<img src="Arch2.png">

Autoencoder- Architecture 3: Changing size of receptive field to 5X5

In [ ]:
#Architecture 3: Changing size of receptive field to 5X5

input_img = Input(shape=(28, 28, 1))  

x = Conv2D(32, (5, 5), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (5, 5), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (5, 5), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (5, 5), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (5, 5), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy',metrics=['mse'])

autoencoder.fit(x_train_noisy, np.concatenate((x_train,x_train),axis=0),
                epochs=12,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/tb', histogram_freq=0, write_graph=False)])
decoded_imgs = autoencoder.predict(x_test_noisy)

plot_digits(x_test,decoded_imgs)

Plot- Autoencoder 3
<img src="Arch3.png">

Autoencoder- Architecture 4: Changing number of hidden layers, adding one more hidden layer

In [ ]:
#Architecture 4: Changing number of hidden layers, adding one more hidden layer

input_img = Input(shape=(28, 28, 1))  

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
#Adding a hidden layer here
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
#Adding a hidden layer here
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy',metrics=['mse'])

autoencoder.fit(x_train_noisy, np.concatenate((x_train,x_train),axis=0),
                epochs=12,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/tb', histogram_freq=0, write_graph=False)])
decoded_imgs = autoencoder.predict(x_test_noisy)

plot_digits(x_test,decoded_imgs)

Plot- Autoencoder 4
<img src="Arch4.png">

Tried other losses like Cosine similarity and mean_squared_logarithmic_error. MSLE worked better than cosine similarity. But for both MSE was much higher than binary_crossentropy loss

Create the MSE curves for the 4 architectures, MSE vs per epoch

In [ ]:
epochs=range(1,13)

#Autoencoder 1
mse_arch1=[0.0210, 0.0072, 0.0059, 0.0051 , 0.0047 , 0.0044 , 0.0041 , 0.0040 ,0.0038 ,0.0037 ,0.0036 ,0.0035 ]
plt.figure()
plt.title('Autoencoder 1- Test MSE over epochs')
plt.xlabel('Number of epochs (# of training samples=120,000*epochs)')
plt.ylabel('Test MSE')
plt.xticks(epochs,epochs)
plt.plot(epochs,mse_arch1)
plt.show()

#Autoencoder 2
mse_arch2=[0.0221 ,0.0088 , 0.0072 , 0.0063 , 0.0058 , 0.0054 ,0.0052 ,0.0050 ,0.0048 ,0.0047 ,0.0046 , 0.0044 ]  #PUT VALUES
plt.figure()
plt.title('Autoencoder 2- Test MSE over epochs')
plt.xlabel('Number of epochs (# of training samples=120,000*epochs)')
plt.ylabel('Test MSE')
plt.xticks(epochs,epochs)
plt.plot(epochs,mse_arch1)
plt.show()

#Autoencoder 3
mse_arch3=[0.0188 , 0.0068 , 0.0055 , 0.0048 ,0.0044 ,0.0041 , 0.0039 ,0.0038 ,0.0036 ,0.0035, 0.0034 ,0.0033 ]
plt.figure()
plt.title('Autoencoder 3- Test MSE over epochs')
plt.xlabel('Number of epochs (# of training samples=120,000*epochs)')
plt.ylabel('Test MSE')
plt.xticks(epochs,epochs)
plt.plot(epochs,mse_arch1)
plt.show()

#Autoencoder 4
mse_arch4=[0.0210, 0.0078 ,  0.0062 , 0.0053 ,0.0048 ,0.0045 ,0.0042 ,0.0040 ,0.0039 ,0.0037 ,0.0036 ,0.0035 ]
plt.figure()
plt.title('Autoencoder 4- Test MSE over epochs')
plt.xlabel('Number of epochs (# of training samples=120,000*epochs)')
plt.ylabel('Test MSE')
plt.xticks(epochs,epochs)
plt.plot(epochs,mse_arch1)
plt.show()

#All together
plt.figure()
label=['Autoencoder 1','Autoencoder 2','Autoencoder 3','Autoencoder 4']
plt.figure()
plt.title('All autoencoders - Test MSE over epochs')
plt.xlabel('Number of epochs (# of training samples=120,000*epochs)')
plt.ylabel('Test MSE')
plt.xticks(epochs,epochs)
plt.plot(epochs,mse_arch1,alpha=1.0,marker='o')
plt.plot(epochs,mse_arch2,alpha=1.0,marker='.')
plt.plot(epochs,mse_arch3,alpha=1.0,marker='*')
plt.plot(epochs,mse_arch4,alpha=1.0,marker=',')
plt.legend(label,loc='best')
plt.show()

Plot MSE vs #epochs for Autoencoder 1,2,3,4. Plot 5 is MSEs plotted for all the 4 autoencoders
<img src="MSE_Arch1.png">
<img src="MSE_Arch2.png">
<img src="MSE_Arch3.png">
<img src="MSE_Arch4.png">
<img src="MSE_All_Arch.png">

As seen from plot, Architecture 2(reduced number of filters per layer) works worst among these. MSE for architecture 1,4 is comparable. Architecture 3 (larger receptive field) works slightly better than 1,4